# Лабораторная работа 5. Разработка единого шаблона предварительной обработки данных

## Подключение библиотек

In [33]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Загрузка данных и разделение на матрицу признаков и зависимую переменную

In [34]:
dataset = pd.read_csv('/content/data5.csv')
dataset.head()

,Genre,Age,Income,Favorite
0,Action,28,60000,Yes
1,Adventure,35,65000,No
2,Comedy,30,55000,Yes
3,Drama,32,58000,Yes
4,Horror,26,52000,No


In [35]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 3].values
print ("Матрица признаков"); print(X)
print ("Зависимая переменная"); print(y)

Матрица признаков
[['Action' 28 60000]
 ['Adventure' 35 65000]
 ['Comedy' 30 55000]
 ['Drama' 32 58000]
 ['Horror' 26 52000]
 ['Sci-Fi' 37 68000]
 ['Thriller' 31 59000]
 ['Documentary' 33 62000]
 ['Animation' 29 56000]
 ['Mystery' 34 64000]]
Зависимая переменная
['Yes' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes' 'Yes' 'No']


### 3. Обработка пропущенных значений

In [36]:
# устаревший подход
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values= np.NAN, strategy= 'mean', fill_value=None, verbose=0, copy=True)
imputer = imputer.fit(X[:, 1:3])
X_without_nan = X.copy()
X_without_nan[:, 1:3] = imputer.transform(X[:, 1:3])
print(X_without_nan)

[['Action' 28.0 60000.0]
 ['Adventure' 35.0 65000.0]
 ['Comedy' 30.0 55000.0]
 ['Drama' 32.0 58000.0]
 ['Horror' 26.0 52000.0]
 ['Sci-Fi' 37.0 68000.0]
 ['Thriller' 31.0 59000.0]
 ['Documentary' 33.0 62000.0]
 ['Animation' 29.0 56000.0]
 ['Mystery' 34.0 64000.0]]


/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:382: FutureWarning: The 'verbose' parameter was deprecated in version 1.1 and will be removed in 1.3. A warning will always be raised upon the removal of empty columns in the future version.
  warnings.warn(


In [37]:
# Новая версия класса-трансформера, предыдущая Imputer - устарела
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer = imputer.fit(X[:, 1:3])
X_without_nan = X.copy()
X_without_nan[:, 1:3] = imputer.transform(X[:, 1:3])
X_without_nan

array([['Action', 28.0, 60000.0],
       ['Adventure', 35.0, 65000.0],
       ['Comedy', 30.0, 55000.0],
       ['Drama', 32.0, 58000.0],
       ['Horror', 26.0, 52000.0],
       ['Sci-Fi', 37.0, 68000.0],
       ['Thriller', 31.0, 59000.0],
       ['Documentary', 33.0, 62000.0],
       ['Animation', 29.0, 56000.0],
       ['Mystery', 34.0, 64000.0]], dtype=object)

### 4. Обработка категориальных данных

#### 4.1 Замена категории кодом (LabelEncoder)

In [38]:
from sklearn.preprocessing import LabelEncoder
labelencoder_y = LabelEncoder()
print("Зависимая переменная до обработки")
print(y)
y = labelencoder_y.fit_transform(y)
print("Зависимая переменная после обработки")
print(y)

Зависимая переменная до обработки
['Yes' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes' 'Yes' 'No']
Зависимая переменная после обработки
[1 0 1 1 0 1 0 1 1 0]


#### 4.2 Применение OneHotEncoder

In [39]:
# создаем копию "грязного" объекта: спропусками и некодированными категориями
X_dirty = X.copy()
X_dirty

array([['Action', 28, 60000],
       ['Adventure', 35, 65000],
       ['Comedy', 30, 55000],
       ['Drama', 32, 58000],
       ['Horror', 26, 52000],
       ['Sci-Fi', 37, 68000],
       ['Thriller', 31, 59000],
       ['Documentary', 33, 62000],
       ['Animation', 29, 56000],
       ['Mystery', 34, 64000]], dtype=object)

In [40]:
# Современный метод трансформации признаков
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# создаем список трансформеров
transformers = [
    ('onehot', OneHotEncoder(), [0]),
    ('imp', SimpleImputer(), [1, 2])
]

# Создаем объект ColumnTransformer и передаем ему список трансформеров
ct = ColumnTransformer(transformers)

# Выполняем трансформацию признаков
X_transformed = ct.fit_transform(X_dirty)
print(X_transformed.shape)
X_transformed

(10, 12)


<10x12 sparse matrix of type '<class 'numpy.float64'>'
	with 30 stored elements in Compressed Sparse Row format>